# Structured Output Example 1:  Calendar Event

In [ ]:
from openai import OpenAI
from pydantic import BaseModel
import os
from dotenv import load_dotenv
load_dotenv()

token = os.getenv("OPENAI_KEY")

client = OpenAI(api_key=token)

class CalendarEvent(BaseModel):
    name: str
    date: str
    participants: list[str]

response = client.responses.parse(
    model="gpt-4o-2024-08-06",
    input=[
        {"role": "system", "content": "Extract the event information."},
        {
            "role": "user",
            "content": "Matt and Alex and Tom are having a meeeting at 3pm next Tuesday in the library",
        },
    ],
    text_format=CalendarEvent,
)

event = response.output_parsed
print(event)


name='Meeting in the Library' date='Next Tuesday at 3pm' participants=['Matt', 'Alex', 'Tom']


# Structured Output Example 2: Ticket Data

In [ ]:
from openai import OpenAI
from pydantic import BaseModel
from typing import Literal
import os
from dotenv import load_dotenv
load_dotenv()

token = os.getenv("OPENAI_KEY")

client = OpenAI(api_key=token)

class SportingEvent(BaseModel):
    sport: Literal["soccer", "basketball", "football", "other"]
    time: Literal["moring", "afternoon", "night"]
    summary: str

response = client.responses.parse(
    model="gpt-4o",
    input=[
        {"role": "system", "content": "Extract structured event data"},
        {
            "role": "user",
            "content": (
                """
                The blockbuster deals that defined last year's sprint to the NBA trade deadline have been replaced by injuries to All-Stars: the Dallas Mavericks' Anthony Davis, Milwaukee Bucks' Giannis Antetokounmpo and Memphis Grizzlies' Ja Morant.

                The injuries and lukewarm interest in Davis and Morant could remove two top trade candidates from the board until the offseason.

                As for Antetokounmpo, team sources told ESPN's Jamal Collier earlier this month that the Bucks are actively looking for trades to improve around their two-time MVP. But with Antetokounmpo out for an extended period because of a calf injury, will Milwaukee stay cautious and regroup in the offseason? Or will Milwaukee unexpectedly start entertaining offers for him? Meanwhile, Jimmy Butler III is not a trade target, but his season-ending ACL tear could play a major role in what direction the Golden State Warriors choose. His teammate, Jonathan Kuminga, has asked to be moved.

                The season has had just one trade -- Trae Young was dealt to the Washington Wizards on Jan. 7 -- but that does not imply an inactive deadline. In the previous two years, 77% of the trades occurred in the week leading up to the deadline.

                As for what to expect by Feb. 5, let's explore each franchise's options at the deadline, starting with two superstars at the center of trade season.
                """
            ),
        },
    ],
    text_format=SportingEvent,
)

ticket = response.output_parsed
print(ticket)

sport='basketball' time='afternoon' summary="The NBA trade deadline is overshadowed by injuries to key players like Anthony Davis, Giannis Antetokounmpo, and Ja Morant, reducing potential trade activities. The Milwaukee Bucks are exploring trades to support Antetokounmpo despite his calf injury, while Golden State's strategy might change following Jimmy Butler III's season-ending injury. Trae Young was recently traded to the Wizards, indicating potential last-minute deals closer to the deadline."


# Tool Calling Example 1: Weather Data

In [ ]:
from openai import OpenAI
from typing import Literal
import json
import os
from dotenv import load_dotenv
load_dotenv()

token = os.getenv("OPENAI_KEY")

client = OpenAI(api_key=token)

# --- Your tools (real functions) ---
def get_weather(city: str, unit: Literal["celsius", "fahrenheit"]) -> dict:
    # Replace with real API call if you want
    return {"city": city, "unit": unit, "temperature": 22, "condition": "sunny"}

def create_calendar_event(name: str, date: str, participants: list[str]) -> dict:
    return {"status": "created", "name": name, "date": date, "participants": participants}

# --- Tool schemas exposed to the model ---
tools = [
    {
        "type": "function",
        "name": "get_weather",
        "description": "Get the current weather for a city",
        "parameters": {
            "type": "object",
            "properties": {
                "city": {"type": "string"},
                "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
            },
            "required": ["city", "unit"],
            "additionalProperties": False,
        },
    },
    {
        "type": "function",
        "name": "create_calendar_event",
        "description": "Create a calendar event",
        "parameters": {
            "type": "object",
            "properties": {
                "name": {"type": "string"},
                "date": {"type": "string"},
                "participants": {"type": "array", "items": {"type": "string"}},
            },
            "required": ["name", "date", "participants"],
            "additionalProperties": False,
        },
    },
]

prompt = "Hi how are you - whats the weather in Miami"

# 1) Start the turn
input_list = [{"role": "user", "content": prompt}]

resp = client.responses.create(
    model="gpt-4o-mini",
    tools=tools,
    input=input_list,
)

# 2) Add model outputs (including any function_call items) to the running input
input_list += resp.output

# 3) Execute any tool calls and append function_call_output items
for item in resp.output:
    if item.type == "function_call":
        args = json.loads(item.arguments)

        print("MODEL CHOSE TOOL:", item.name)
        print("ARGS:", args)

        if item.name == "get_weather":
            result = get_weather(**args)
        elif item.name == "create_calendar_event":
            result = create_calendar_event(**args)
        else:
            raise ValueError(f"Unknown tool: {item.name}")

        input_list.append(
            {
                "type": "function_call_output",
                "call_id": item.call_id,
                "output": json.dumps(result),
            }
        )

# 4) Ask the model again, now that it has the tool output(s)
final = client.responses.create(
    model="gpt-4o-mini",
    tools=tools,
    input=input_list,
)

print("\nFINAL MODEL OUTPUT:")
print(final.output_text)

MODEL CHOSE TOOL: get_weather
ARGS: {'city': 'Miami', 'unit': 'celsius'}

FINAL MODEL OUTPUT:
I'm doing well, thanks for asking! The weather in Miami is currently 22°C and sunny. How can I assist you further?


# Tool Calling Example 2: Weather Data - Actual Tool

In [ ]:
from openai import OpenAI
from typing import Literal
import json
import requests
import os
from dotenv import load_dotenv
load_dotenv()

token_one = os.getenv("OPENAI_KEY")
WEATHERAPI_KEY = os.getenv("WEATHERAPI_KEY")

client = OpenAI(api_key=token_one)

# ---------- REAL WEATHER TOOL (Open-Meteo) ----------
def get_weather(
    city: str,
    unit: Literal["celsius", "fahrenheit"],
    debug: bool = False,
) -> dict:
    """
    WeatherAPI.com realtime/current endpoint.

    Endpoint:
      https://api.weatherapi.com/v1/current.json?key=...&q=...

    Returns:
      location + current fields (temp_c/temp_f, condition.text, wind, humidity, etc.)
    """
    url = "https://api.weatherapi.com/v1/current.json"
    params = {
        "key": WEATHERAPI_KEY,
        "q": city,          # e.g. "Durham, NC" or zip or lat,lon
        "aqi": "no",        # optional; keeps response smaller
    }

    r = requests.get(url, params=params, timeout=20)
    r.raise_for_status()
    data = r.json()

    loc = data.get("location", {})
    cur = data.get("current", {})
    cond = (cur.get("condition") or {})

    out = {
        "resolved_location": {
            "name": loc.get("name"),
            "region": loc.get("region"),
            "country": loc.get("country"),
            "lat": loc.get("lat"),
            "lon": loc.get("lon"),
            "localtime": loc.get("localtime"),
        },
        "current": {
            "last_updated": cur.get("last_updated"),
            "temp_c": cur.get("temp_c"),
            "temp_f": cur.get("temp_f"),
            "feelslike_c": cur.get("feelslike_c"),
            "feelslike_f": cur.get("feelslike_f"),
            "humidity": cur.get("humidity"),
            "cloud": cur.get("cloud"),
            "is_day": cur.get("is_day"),
            "uv": cur.get("uv"),
            "wind_mph": cur.get("wind_mph"),
            "wind_kph": cur.get("wind_kph"),
            "wind_degree": cur.get("wind_degree"),
            "wind_dir": cur.get("wind_dir"),
            "gust_mph": cur.get("gust_mph"),
            "gust_kph": cur.get("gust_kph"),
            "pressure_mb": cur.get("pressure_mb"),
            "pressure_in": cur.get("pressure_in"),
            "precip_mm": cur.get("precip_mm"),
            "precip_in": cur.get("precip_in"),
            "condition": {
                "text": cond.get("text"),
                "icon": cond.get("icon"),
                "code": cond.get("code"),
            },
            # convenience: pick one display temp based on unit
            "display_temp": cur.get("temp_f") if unit == "fahrenheit" else cur.get("temp_c"),
            "display_unit": "F" if unit == "fahrenheit" else "C",
        },
        "source": "weatherapi.com",
    }

    if debug:
        out["_debug"] = {"request_url": r.url}

    return out


def create_calendar_event(name: str, date: str, participants: list[str]) -> dict:
    return {"status": "created", "name": name, "date": date, "participants": participants}


tools = [
    {
        "type": "function",
        "name": "get_weather",
        "description": "Get current weather for a city using a live weather API.",
        "parameters": {
            "type": "object",
            "properties": {
                "city": {"type": "string", "description": "City name, e.g. 'Durham, NC'"},
                "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
            },
            "required": ["city", "unit"],
            "additionalProperties": False,
        },
    },
    {
        "type": "function",
        "name": "create_calendar_event",
        "description": "Create a calendar event",
        "parameters": {
            "type": "object",
            "properties": {
                "name": {"type": "string"},
                "date": {"type": "string"},
                "participants": {"type": "array", "items": {"type": "string"}},
            },
            "required": ["name", "date", "participants"],
            "additionalProperties": False,
        },
    },
]

prompt = """Alice and Bob are meeting in Miami this Friday. They want to know if the weather will be good.
Provide a quick summary for them, including what to wear"""

#prompt = "I need to schedule a meeting for tomorrow at 3 with Bob and Sally"


# --- Responses API loop (your working pattern) ---
input_list = [{"role": "user", "content": prompt}]

resp = client.responses.create(
    model="gpt-4o-mini-2024-07-18",
    tools=tools,
    tool_choice="auto",
    input=input_list,
)

print(resp)
# Add model outputs to context
input_list += resp.output

# Execute tool calls and append function_call_output items
for item in resp.output:
    if item.type == "function_call":
        args = json.loads(item.arguments)
        print("MODEL CHOSE TOOL:", item.name)
        print("ARGS:", args)

        if item.name == "get_weather":
            result = get_weather(**args)
        elif item.name == "create_calendar_event":
            result = create_calendar_event(**args)
        else:
            raise ValueError(f"Unknown tool: {item.name}")

        input_list.append(
            {
                "type": "function_call_output",
                "call_id": item.call_id,
                "output": json.dumps(result),
            }
        )

final = client.responses.create(
    model="gpt-4o-mini-2024-07-18",
    tools=tools,
    input=input_list,
)

print("\nFINAL MODEL OUTPUT:")
print(final.output_text)

Response(id='resp_09d2dc121fa3eaee00697924b6c6cc81909a1b093fc47afa3d', created_at=1769546934.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='gpt-4o-mini-2024-07-18', object='response', output=[ResponseFunctionToolCall(arguments='{"city":"Miami, FL","unit":"fahrenheit"}', call_id='call_sPPDvQpVVUTYYOKBsEqxxiDl', name='get_weather', type='function_call', id='fc_09d2dc121fa3eaee00697924b7597c819096c48d40b2f53b20', status='completed')], parallel_tool_calls=True, temperature=1.0, tool_choice='auto', tools=[FunctionTool(name='get_weather', parameters={'type': 'object', 'properties': {'city': {'type': 'string', 'description': "City name, e.g. 'Durham, NC'"}, 'unit': {'type': 'string', 'enum': ['celsius', 'fahrenheit']}}, 'required': ['city', 'unit'], 'additionalProperties': False}, strict=True, type='function', description='Get current weather for a city using a live weather API.'), FunctionTool(name='create_calendar_event', parameters={'type': 'object', 'proper

# Assignment 3

In [8]:
from openai import OpenAI
from typing import Literal, List
import json
import requests
import os
from dotenv import load_dotenv

load_dotenv()
token_one = os.getenv("OPENAI_KEY")
client = OpenAI(api_key=token_one)

# Reference
"""
    OpenBreweryDB endpoint:
      https://api.openbrewerydb.org/v1/breweries

    Returns:
      List of breweries in a given city or state
"""
def get_breweries(
    city: str,
    state: str = None,
    limit: int = 3,
    debug: bool = False,
) -> dict:
    url = "https://api.openbrewerydb.org/v1/breweries"
    params = {
        "by_city": city,
        "per_page": limit,
    }

    if state:
        params["by_state"] = state

    r = requests.get(url, params=params, timeout=30)
    r.raise_for_status()
    data = r.json()

    breweries = []
    for b in data:
        breweries.append({
            "name": b.get("name"),
            "brewery_type": b.get("brewery_type"),
            "address": ", ".join(
                filter(None, [
                    b.get("street"),
                    b.get("city"),
                    b.get("state"),
                ])
            ),
            "website": b.get("website_url"),
        })

    out = {
        "city": city,
        "state": state,
        "count": len(breweries),
        "breweries": breweries,
        "source": "openbrewerydb.org",
    }

    if debug:
        out["_debug"] = {"request_url": r.url}

    return out


def save_recommendation(title: str, notes: str, people: List[str]) -> dict:
    return {
        "status": "saved",
        "title": title,
        "notes": notes,
        "shared_with": people,
    }

# Tools
'''Gets the breweries based on input, and/or saves a bewery recommendation'''
tools = [
    {
        "type": "function",
        "name": "get_breweries",
        "description": "Find breweries in a given city using OpenBreweryDB.",
        "parameters": {
            "type": "object",
            "properties": {
                "city": {"type": "string", "description": "City name, e.g. 'Durham'"},
                "state": {"type": "string", "description": "State name, e.g. 'North Carolina'"},
                "limit": {"type": "integer", "description": "Max number of breweries"},
            },
            "required": ["city"],
            "additionalProperties": False,
        },
    },
    {
        "type": "function",
        "name": "save_recommendation",
        "description": "Save a recommendation or plan for later reference.",
        "parameters": {
            "type": "object",
            "properties": {
                "title": {"type": "string"},
                "notes": {"type": "string"},
                "people": {"type": "array", "items": {"type": "string"}},
            },
            "required": ["title", "notes", "people"],
            "additionalProperties": False,
        },
    },
]

prompt = """
Argyle is a brewery owner who wants to know the competition in my area. 
What are some notable breweries in the Durham, NC area?
"""

input_list = [{"role": "user", "content": prompt}]

resp = client.responses.create(
    model="gpt-4o-mini-2024-07-18",
    tools=tools,
    tool_choice="auto",
    input=input_list,
)

print(resp)

input_list += resp.output

for item in resp.output:
    if item.type == "function_call":
        args = json.loads(item.arguments)
        print("MODEL CHOSE TOOL:", item.name)
        print("ARGS:", args)

        if item.name == "get_breweries":
            result = get_breweries(**args)
        elif item.name == "save_recommendation":
            result = save_recommendation(**args)
        else:
            raise ValueError(f"Unknown tool: {item.name}")

        input_list.append(
            {
                "type": "function_call_output",
                "call_id": item.call_id,
                "output": json.dumps(result),
            }
        )

final = client.responses.create(
    model="gpt-4o-mini-2024-07-18",
    tools=tools,
    input=input_list,
)

print("\nFINAL OUTPUT:")
print(final.output_text)


Response(id='resp_0e969e1220b998f900697fcd49dfb4819d9aa6ddaf11ab4da7', created_at=1769983305.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='gpt-4o-mini-2024-07-18', object='response', output=[ResponseFunctionToolCall(arguments='{"city":"Durham","state":"North Carolina","limit":5}', call_id='call_s8WNg41dW5KPOICD68JPpl82', name='get_breweries', type='function_call', id='fc_0e969e1220b998f900697fcd4ab878819d843be435c67f8985', status='completed')], parallel_tool_calls=True, temperature=1.0, tool_choice='auto', tools=[FunctionTool(name='get_breweries', parameters={'type': 'object', 'properties': {'city': {'type': 'string', 'description': "City name, e.g. 'Durham'"}, 'state': {'type': 'string', 'description': "State name, e.g. 'North Carolina'"}, 'limit': {'type': 'integer', 'description': 'Max number of breweries'}}, 'required': ['city', 'state', 'limit'], 'additionalProperties': False}, strict=True, type='function', description='Find breweries in a given ci